In [7]:
# Import libraries
import re
import nltk
import string
import pyforest
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [8]:
# Read the CSV file
columns  = ["sentiment", "ID", "datetime", "query", "username", "text"]
df = pd.read_csv('sentiment_dataset.csv', delimiter=',', encoding = "ISO-8859-1", names = columns)
df.head()

<IPython.core.display.Javascript object>

,sentiment,ID,datetime,query,username,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
# Extract sentiment and text column, they will be relevant in this analysis
df = df[['sentiment','text']]
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [10]:
# Fuction that performs cleaning using re library. 
# It removes uppercase, brackets, links, punctuation etc
def cleaning(a):
    a = str(a).lower()
    a = re.sub('\[.*?\]', '', a)
    a = re.sub('[%s]' % re.escape(string.punctuation), '', a)
    a = re.sub('\n', '', a)
    a = re.sub('https?://\S+|www\.\S+', '', a)
    a = re.sub('<.*?>+', '', a)
    a = re.sub('\w*\d\w*', '', a)
    return a

df['text'] = df['text'].apply(cleaning)
df.head()

,sentiment,text
0,0,switchfoot awww thats a bummer you shoulda ...
1,0,is upset that he cant update his facebook by t...
2,0,kenichan i dived many times for the ball manag...
3,0,my whole body feels itchy and like its on fire
4,0,nationwideclass no its not behaving at all im ...


In [11]:
# Checking for balance
df['sentiment'].value_counts()

0    800000
4    800000
Name: sentiment, dtype: int64

In [12]:
# Removing stop words i.e., the, is, and, or, in, this etc
s_words = stopwords.words('english')

def removing(text):
    a = ' '.join(i for i in text.split(' ') if i not in s_words)
    return a
    
df['text'] = df['text'].apply(removing)
df.head()

,sentiment,text
0,0,switchfoot awww thats bummer shoulda got da...
1,0,upset cant update facebook texting might cry r...
2,0,kenichan dived many times ball managed save ...
3,0,whole body feels itchy like fire
4,0,nationwideclass behaving im mad cant see


In [13]:
# Change 0 to negative and 4 to positive
class_dict = {0:'negative', 4:'positive'}
df['sentiment'] = df['sentiment'].apply(lambda x:  class_dict[x])
df.head()

,sentiment,text
0,negative,switchfoot awww thats bummer shoulda got da...
1,negative,upset cant update facebook texting might cry r...
2,negative,kenichan dived many times ball managed save ...
3,negative,whole body feels itchy like fire
4,negative,nationwideclass behaving im mad cant see


In [14]:
# Stemming words that have same meaning
stemmer = nltk.SnowballStemmer("english")

def stemming(text):
    text = ' '.join(stemmer.stem(i) for i in text.split(' '))
    return text

df['text'] = df['text'].apply(stemming)
df.head()

,sentiment,text
0,negative,switchfoot awww that bummer shoulda got dav...
1,negative,upset cant updat facebook text might cri resul...
2,negative,kenichan dive mani time ball manag save rest...
3,negative,whole bodi feel itchi like fire
4,negative,nationwideclass behav im mad cant see


In [15]:
# Split the data to train data and test data
X = df['text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
# Using Logistic Regression
clf = LogisticRegression(penalty="l2", random_state=42, C=0.01)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Training Accuracy:", clf.score(X_train, y_train))
print("Testing Accuracy:", clf.score(X_test, y_pred))

ValueError: could not convert string to float: 'nathyeah  '